In [31]:
#read pdf
#clean data
#write to excel
#clean manually
#convert to cldf

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
from io import StringIO

import re
import pandas as pd

def get_pdf_file_content(path_to_pdf):
    resource_manager = PDFResourceManager(caching=True)
    out_text = StringIO()
    laParams = LAParams()
    text_converter = TextConverter(resource_manager, out_text,laparams=laParams)
    fp = open(path_to_pdf, 'rb')
    interpreter = PDFPageInterpreter(resource_manager,text_converter)
    for page in PDFPage.get_pages(fp,pagenos=set(),maxpages=0,password="",caching=True,check_extractable=True):
        interpreter.process_page(page)
    text= out_text.getvalue()
    
    fp.close()
    text_converter.close()
    out_text.close()
    
    return text


def clean1():
    mri = get_pdf_file_content("duval_thesis_vol3-1-40.pdf")
    mri2=[]
    for i in mri.split("\n\n"):
        if re.search('\x0c[1-9]', i) or not re.search('[a-zA-Z]', i):
            continue
        else:
            mri2.append(i)
    return mri2

def clean2():
    i = iter(clean1())
    counter=0
    return pd.DataFrame(list(zip(i, i)), columns=["mri","en"])

def clean3():
    mri = clean2()
    for col in ["mri", "en"]:
        mri[col] = [re.sub("[1-9]","",i) for i in mri[col]]
        mri[col] = [re.sub(" *?\[.*?\]? *?","",i) for i in mri[col]]
        mri[col] = [re.sub(" *?\(.*?\) *","",i) for i in mri[col]]
        mri[col] = [re.sub("[\*\?\"\.'\(\)0\;\_\]\}·\~]","",i) for i in mri[col]]
    
    mri["mri"] = [i.split("\n") for i in mri["mri"]]
    mri["en"] = [i.split("\n") for i in mri["en"]]
    
    mri["en"] = [None if i[0]=="?" else i for i in mri["en"]]
    mri["en"] = [[j for j in i if j!=" "] for i in mri["en"]]
    mri = mri.dropna()
    mri["lenmri"] = list(map(len, mri["mri"]))
    mri["lenen"] = list(map(len, mri["en"]))
    #print("Following rows have a different number of elements. Trim them manually.")
    #for k, (i,j) in enumerate(zip(mri["lenmri"], mri["lenen"])):
     #   if i !=j:
      #      print(k)
            
    mri.to_csv("duv95_coarse.csv", encoding="utf_8_sig", index=False)
    #mri = mri.explode(["mri", "en"])
       
    return mri

clean3()

,mri,en,lenmri,lenen
0,"[PL , -b , Z , > , v ]",[PLACE NAMES ],5,1
1,[],[Iti ],1,1
2,"[Abisinia , Aearana , Aferika , Afrika , Aharo...","[Abysinnia , Ireland , Africa , Africa , , Asi...",37,36
3,"[Arena , Arepia , Arikehanara , Aripa , Armeni...","[Aliens Creek, Arabia , Alexandria, Alps, Arm...",38,37
4,"[Britania , Britannia ]","[Britain , Britain ]",2,2
...,...,...,...,...
65,"[Korowi , Kotepere , Koteriki , Kotiere , Koti...","[Leonida , Lisiniu ]",10,2
66,"[Ma, Maata , Maehe , Mahameta , Maharere , Mah...","[Clovis , Godfrey , Clotilde , Chrysostom , Cr...",27,7
67,"[Leonidas , Licinius ]","[Mary , Martha , Marsh , Mahomed , Marcellus ,...",2,26
68,"[Makarita , Makehe , Makehe , Makentiu , Maker...","[Margaret , Maxentius , Marcus , Maxentius , M...",43,30


In [13]:
x = ["a", "", "b"]
[]

False

In [ ]:
import re
re.sub(" *?\(.*?\) *","","Amsterdam (Holland)")

In [ ]:
set("".join(mri["mri"]))

In [ ]:
import pandas as pd
import re

mri = pd.read_csv("mri1.csv")
mri["en"] = [None if i[0]=="?" else i for i in mri["en"]]
mri = mri.dropna()
for col in ["mri", "en"]:
    mri[col] = [re.sub("[1-9]","",i) for i in mri[col]]
    mri[col] = [re.sub(" *?\[.*?\]? *?","",i) for i in mri[col]]
    mri[col] = [re.sub(" *?\(.*?\) *","",i) for i in mri[col]]
    mri[col] = [re.sub("[\*\?\"\.'\(\)0\;\_\]\}·\~]","",i) for i in mri[col]]
mri.head(50)

In [ ]:
import pandas as pd
mri = pd.read_excel("mripdf_pre.xlsx")
#mri["lenmri"] = [len(i.split("\n")) for i in mri["mri"]]
#mri["lenen"] = [len(i.split("\n")) for i in mri["en"]]
mri["mri"] = [i.split("\n") for i in mri["mri"]]
mri["en"] = [i.split("\n") for i in mri["en"]]
mri = mri.explode(["mri", "en"])
#for k, (i,j) in enumerate(zip(mri["lenmri"], mri["lenen"])):
 #   if i !=j:
  #      print(k)
mri
#for i,j in mri.iterrows():
 #   if i==15:
   #     for k,l in zip(j["mri"].split("\n"), j["en"].split("\n")):
  #          print(k,l)

In [87]:
mri.to_csv("mri2.csv", encoding="utf_8_sig", index=False)

In [ ]:
#doesn't really work
import tabula

# Read PDF into list of DataFrame
dataframe = tabula.read_pdf("duval_thesis_vol3.pdf", pages='all')

# Read remote PDF into list of DataFrame
#dataframe_2 = tabula.read_pdf("https://github.com/tabulapdf/tabula-java/raw/master/src/test/resources/technology/tabula/arabic.pdf")

# Convert PDF into CSV file
tabula.convert_into("duval_thesis_vol3.pdf", "output.csv", output_format="csv", pages='all')

In [2]:
#turns pdf 2 txt but tricky to put that text into a df
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
from io import StringIO

def get_pdf_file_content(path_to_pdf):
    resource_manager = PDFResourceManager(caching=True)
    out_text = StringIO()
    laParams = LAParams()
    text_converter = TextConverter(resource_manager, out_text,laparams=laParams)
    fp = open(path_to_pdf, 'rb')
    interpreter = PDFPageInterpreter(resource_manager,text_converter)
    for page in PDFPage.get_pages(fp,pagenos=set(),maxpages=0,password="",caching=True,check_extractable=True):
        interpreter.process_page(page)
    text= out_text.getvalue()
    
    fp.close()
    text_converter.close()
    out_text.close()
    
    return text

mri = get_pdf_file_content("duval_thesis_vol3-1-40.pdf")

In [ ]:
mri.split("\n\n")

In [ ]:
i = iter(mri2)
counter=0

In [ ]:
import pandas as pd
mri3 = pd.DataFrame(list(zip(i, i)), columns=["mri","en"])
mri3

In [ ]:
mri3.to_csv("duvalmri.csv", encoding="utf_8_sig", index=False)

In [ ]:
counter+=1
next(i)

In [ ]:
type(mri2)

In [ ]:
from re import search

mri2=[]
for i in mri.split("\n\n"):
    if search('\x0c[1-9]', i) or not search('[a-zA-Z]', i):
        continue
    else:
        mri2.append(i)
mri2

In [ ]:
mri

In [ ]:
#doesn't work b/c "poppler" has a bug
#https://www.geeksforgeeks.org/python-reading-contents-of-pdf-using-ocr-optical-character-recognition/
# Import libraries
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os

# Path of the pdf
PDF_file = "duval_thesis_vol3.pdf"

'''
Part #1 : Converting PDF to images
'''

# Store all the pages of the PDF in a variable
pages = convert_from_path(PDF_file, 189)#, poppler_path=r'C:\Users\Viktor\OneDrive\PhD cloud\Vorgehensweisen\loanpy11\tests_phd\test_adapter\maori-en\maori_en2\poppler-0.68.0_x86\bin')

# Counter to store images of each page of PDF to image
image_counter = 1

# Iterate through all the pages stored above
for page in pages:

	# Declaring filename for each page of PDF as JPG
	# For each page, filename will be:
	# PDF page 1 -> page_1.jpg
	# PDF page 2 -> page_2.jpg
	# PDF page 3 -> page_3.jpg
	# ....
	# PDF page n -> page_n.jpg
	filename = "page_"+str(image_counter)+".jpg"
	
	# Save the image of the page in system
	page.save(fr"pdfimages\{filename}", 'JPEG')

	# Increment the counter to update filename
	image_counter = image_counter + 1

'''
Part #2 - Recognizing text from the images using OCR
'''

# Variable to get count of total number of pages
filelimit = image_counter-1

# Creating a text file to write the output
outfile = "out_text.txt"

# Open the file in append mode so that
# All contents of all images are added to the same file
f = open(outfile, "a")

# Iterate from 1 to total number of pages
for i in range(1, filelimit + 1):

	# Set filename to recognize text from
	# Again, these files will be:
	# page_1.jpg
	# page_2.jpg
	# ....
	# page_n.jpg
	filename = "page_"+str(i)+".jpg"
		
	# Recognize the text as string in image using pytesserct
	text = str(((pytesseract.image_to_string(Image.open(filename)))))

	# The recognized text is stored in variable text
	# Any string processing may be applied on text
	# Here, basic formatting has been done:
	# In many PDFs, at line ending, if a word can't
	# be written fully, a 'hyphen' is added.
	# The rest of the word is written in the next line
	# Eg: This is a sample text this word here GeeksF-
	# orGeeks is half on first line, remaining on next.
	# To remove this, we replace every '-\n' to ''.
	text = text.replace('-\n', '')	

	# Finally, write the processed text to the file.
	f.write(text)

# Close the file after writing all the text.
f.close()
